<a href="https://github.com/Bae-ChangHyun"><img src="https://img.shields.io/badge/Github-000000?style=flat&logo=github&logoColor=ffffff&labelColor=000000&link=https%3A%2F%2Fgithub.com%2FBae-ChangHyun"/></a> <br>
<a href="https://changsroad.tistory.com/"><img src="https://img.shields.io/badge/Tistory-f44336?style=flat&logo=tistory&logoColor=ffffff&link=https%3A%2F%2Fchangsroad.tistory.com%2F"/></a> <br>
<a href="mailto:matthew624@naver.com"><img src="https://img.shields.io/badge/Naver-03C75A?style=flat&logo=naver&logoColor=ffffff&link=mailto%3Amatthew624%40naver.com"/></a><br>
<a href="https://www.kaggle.com/competitions/spaceship-titanic" target="_blank"><img align="left" alt="Kaggle" title="Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Spaceship titanic 예측

## 1. 라이브러리, 데이터 불러오기

In [1]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import os
import random

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearnex import patch_sklearn
patch_sklearn()
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
# hyper-parameter tuning을 위한 라이브러리
from sklearn.model_selection import GridSearchCV
import optuna
from optuna.samplers import TPESampler

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
# 데이터를 불러옵니다.
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

## 2. EDA

In [3]:
# 1. 결측치 체크 --> 특정 column이 많은 결측치를 포함하고 있는지!
null_row=train.isnull().any(axis=1) # boolean mask
train[null_row] # row에 하나라도 결측치가 있는 row
# 2. dtype이 object인 column들 체크 (str)
cat_cols=train.columns[train.dtypes == 'object'] # 카테고리형 변수
num_cols=train.columns[~(train.dtypes == 'object')] # 수치형 변수
# 3. class imbalance 체크!
train.Transported.value_counts()
train.HomePlanet.value_counts()
train.Destination.value_counts()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
10,0008_02,Europa,True,B/1/P,TRAPPIST-1e,34.0,False,0.0,0.0,NaN,0.0,0.0,Altardr Flatic,True
15,0012_01,Earth,False,NaN,TRAPPIST-1e,31.0,False,32.0,0.0,876.0,0.0,0.0,Justie Pooles,False
16,0014_01,Mars,False,F/3/P,55 Cancri e,27.0,False,1286.0,122.0,NaN,0.0,0.0,Flats Eccle,False
23,0020_03,Earth,True,E/0/S,55 Cancri e,29.0,False,0.0,0.0,NaN,0.0,0.0,Mollen Mcfaddennon,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8667,9250_01,Europa,False,E/597/P,TRAPPIST-1e,29.0,False,0.0,2972.0,NaN,28.0,188.0,Chain Reedectied,True
8674,9257_01,NaN,False,F/1892/P,TRAPPIST-1e,13.0,False,39.0,0.0,1085.0,24.0,0.0,Ties Apple,False
8675,9259_01,Earth,NaN,F/1893/P,TRAPPIST-1e,44.0,False,1030.0,1015.0,0.0,11.0,NaN,Annah Gilleyons,True
8684,9274_01,NaN,True,G/1508/P,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Chelsa Bullisey,True


Transported
True     4378
False    4315
Name: count, dtype: int64

HomePlanet
Earth     4602
Europa    2131
Mars      1759
Name: count, dtype: int64

Destination
TRAPPIST-1e      5915
55 Cancri e      1800
PSO J318.5-22     796
Name: count, dtype: int64

In [4]:
train
# passengerId : group_num(4자리, usually family but not always) + id(그룹내에서 2자리)
# HomePlanet, Desitination : categorical feature.
# Cabin (side deck / otherwise)
# CryoSleep, VIP : bool
# Age, RoomService, FoodCourt, ShoppingMall, Spa, VRDeck : 쓴 돈.
# Name : 이름
# Transported(y) : True / False

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [5]:
# VIP에 따른 transported 분포
pd.pivot_table(train,values='Transported' ,index='VIP',aggfunc=['mean'])
train.groupby(['VIP','Transported']).size()

# HomePlanet, VIP에 따른 transported 분포
pd.pivot_table(train,values='Transported' ,index=['HomePlanet','VIP'],aggfunc=['mean'])

,mean
,Transported
VIP,
False,0.506332
True,0.381910


VIP    Transported
False  False          4093
       True           4198
True   False           123
       True             76
dtype: int64

mean
                 Transported
HomePlanet VIP              
Earth      False    0.424337
Europa     False    0.670072
           True     0.488550
Mars       False    0.534180
           True     0.158730

### 3. 전처리

#### 결측치 처리

In [6]:
## TO-DO ##
## group 정보를 passenger_Id로부터 추출해서, group_size가 4 이상인 고객들은 in_large_group 이라는 column에 1, 나머지는 0으로 하는 feature 생성
group = train.PassengerId.apply(lambda x:x[:4])
temp = group.value_counts()
large_group_num = temp[temp >= 4].index
# isin이 0 혹은 1반환 
train['in_large_group'] = group.isin(large_group_num) * 1
# 의미있어보이지 않음..
pd.pivot_table(train,values='Transported' ,index=['in_large_group'],aggfunc=['mean'])
train

,mean
,Transported
in_large_group,
0,0.490742
1,0.585160


,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,in_large_group
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,0
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,0
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,0
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False,0
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True,0
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False,0


In [7]:
train['HomePlanet'].value_counts()

HomePlanet
Earth     4602
Europa    2131
Mars      1759
Name: count, dtype: int64

In [8]:
# 결측치가 있는 column
# 결측치 제거 방법
# 1) cat_cols에 대해서는 mode를 계산해서 채워주고, num_cols에 대해서는 mean을 채워줌
train[cat_cols] = train[cat_cols].apply(lambda col: col.fillna(col.mode()[0]), axis=0) # mode()를 채울 땐 [0]을 추가해줘야 함.
train[num_cols] = train[num_cols].apply(lambda col: col.fillna(col.mean()), axis=0)
train.info()

# categorical feature encoding
train = pd.get_dummies(data=train, columns=['HomePlanet','Destination'])
drop_cols=['PassengerId', 'Cabin', 'Name']
train.drop(drop_cols, axis=1, inplace=True)
train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     8693 non-null   object 
 1   HomePlanet      8693 non-null   object 
 2   CryoSleep       8693 non-null   bool   
 3   Cabin           8693 non-null   object 
 4   Destination     8693 non-null   object 
 5   Age             8693 non-null   float64
 6   VIP             8693 non-null   bool   
 7   RoomService     8693 non-null   float64
 8   FoodCourt       8693 non-null   float64
 9   ShoppingMall    8693 non-null   float64
 10  Spa             8693 non-null   float64
 11  VRDeck          8693 non-null   float64
 12  Name            8693 non-null   object 
 13  Transported     8693 non-null   bool   
 14  in_large_group  8693 non-null   int32  
dtypes: bool(3), float64(6), int32(1), object(5)
memory usage: 806.6+ KB


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,in_large_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False,0,False,True,False,False,False,True
1,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True,0,True,False,False,False,False,True
2,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False,0,False,True,False,False,False,True
3,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False,0,False,True,False,False,False,True
4,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True,0,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,False,41.0,True,0.0,6819.0,0.0,1643.0,74.0,False,0,False,True,False,True,False,False
8689,True,18.0,False,0.0,0.0,0.0,0.0,0.0,False,0,True,False,False,False,True,False
8690,False,26.0,False,0.0,0.0,1872.0,1.0,0.0,True,0,True,False,False,False,False,True
8691,False,32.0,False,0.0,1049.0,0.0,353.0,3235.0,False,0,False,True,False,True,False,False


## 4. 학습 및 평가

### 학습 데이터 분할

In [9]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

X =  train.drop(['Transported'],axis=1)
y =  train['Transported']

X_train, X_val, y_train, y_val = train_test_split(X,y,train_size=0.8,random_state=624)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)

(6954, 15) (6954,) (1739, 15) (1739,)


### Hyper parameter tuning

#### Human search
- n_estimator의 경우도 숫자를 늘려도 큰 변화없어보임. 
- max_depth의 경우 숫자가 커질수록 train score가 올라가지만, valid는 여전함.
- min_samples_leaf의 경우 낮게 설정해도 score 동일

In [10]:
params = {
    'n_estimators': 500,  # 결정 트리의 개수
    'max_depth': 7,    # 각 결정 트리의 최대 깊이 (None으로 설정하면 깊이 제한 없음->오버피팅)
    'min_samples_split': 10,  # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': 5,   # 리프 노드에 있어야 하는 최소 샘플 수
    'max_features': 0.7,  # 각 노드에서 분할에 사용할 특성의 최대 수 ('auto', 'sqrt', 'log2', 또는 정수)
    'random_state': 624,   # 랜덤 시드 (재현성을 위해 고정된 값 사용)
    'n_jobs':-1
}

In [11]:
model = RandomForestClassifier(**params)
model.fit(X_train, y_train)

print("Prediction")
pred_train = model.predict(X_train)
pred_val = model.predict(X_val)

train_score = accuracy_score(y_train, pred_train)
val_score = accuracy_score(y_val, pred_val)

print("Train Score : %.4f" % train_score)
print("Validation Score : %.4f" % val_score)

RandomForestClassifier(max_depth=7, max_features=0.7, min_samples_leaf=5,
                       min_samples_split=10, n_estimators=500, n_jobs=-1,
                       random_state=624)

Prediction
Train Score : 0.8108
Validation Score : 0.7953


#### Gridsearch

In [12]:
params_grid = {
    'n_estimators': [50,100,200,500],  # 결정 트리의 개수
    'max_depth': [6,7,8,15],    # 각 결정 트리의 최대 깊이 (None으로 설정하면 깊이 제한 없음->오버피팅)
    'min_samples_split': [2,10,25],  # 노드를 분할하기 위한 최소 샘플 수
    'min_samples_leaf': [1,5,10],   # 리프 노드에 있어야 하는 최소 샘플 수
    'max_features': [0.5,0.7,0.9],  # 각 노드에서 분할에 사용할 특성의 최대 수 ('auto', 'sqrt', 'log2', 또는 정수)
}

gcv = GridSearchCV(estimator=RandomForestClassifier(random_state=624), param_grid=params_grid, cv=5, n_jobs=-1, verbose=2)

gcv.fit(X_train, y_train)
print("Best Estimator : ", gcv.best_estimator_)

Fitting 5 folds for each of 432 candidates, totalling 2160 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=624),
             n_jobs=-1,
             param_grid={'max_depth': [6, 7, 8, 15],
                         'max_features': [0.5, 0.7, 0.9],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10, 25],
                         'n_estimators': [50, 100, 200, 500]},
             verbose=2)

Best Estimator :  RandomForestClassifier(max_depth=8, max_features=0.5, min_samples_leaf=5,
                       min_samples_split=25, random_state=624)


In [13]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_train)
gcv_pred_val = gcv.predict(X_val)

gcv_train_score = accuracy_score(y_train, gcv_pred_train)
gcv_val_score = accuracy_score(y_val, gcv_pred_val)

print("Train ACC Score : %.4f" % gcv_train_score)
print("Validation ACC Score : %.4f" % gcv_val_score)

Prediction with Best Estimator
Train ACC Score : 0.8159
Validation ACC Score : 0.7987


#### OPtuna

In [18]:
def objective(trial):
    # hyper-parameter
    params_optuna={
        "n_estimators" : trial.suggest_int('n_estimators', 50, 200),
        "max_depth" : 10,
        "min_samples_split" : trial.suggest_categorical('min_samples_split',[2,10,25]),
        "min_samples_leaf" : trial.suggest_int('min_samples_leaf', 1, 10),
        "max_features" : trial.suggest_float('max_features',0.5, 0.8),
        }
    
    model = RandomForestClassifier(**params_optuna)
    
    model.fit(X_train,y_train)
    preds=model.predict(X_val)
    scores=accuracy_score(y_val,preds)

    return scores

In [19]:
K = 5 # Kfold 수
sampler = TPESampler(seed=624)
study = optuna.create_study(direction="maximize", # 최소/최대 어느 방향의 최적값을 구할 건지.
                            sampler=sampler,
                            storage="sqlite:///db.sqlite3",
                            study_name="test2(23.11.28)") 
study.optimize(objective, n_trials=100)

[I 2023-11-28 12:35:44,628] A new study created in RDB with name: test2(23.11.28)
[I 2023-11-28 12:35:44,956] Trial 0 finished with value: 0.8027602070155262 and parameters: {'n_estimators': 65, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.6354105425860979}. Best is trial 0 with value: 0.8027602070155262.
[I 2023-11-28 12:35:45,269] Trial 1 finished with value: 0.8010350776308223 and parameters: {'n_estimators': 53, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 0.6424699194047778}. Best is trial 0 with value: 0.8027602070155262.
[I 2023-11-28 12:35:45,626] Trial 2 finished with value: 0.8027602070155262 and parameters: {'n_estimators': 108, 'min_samples_split': 25, 'min_samples_leaf': 6, 'max_features': 0.6910121372583131}. Best is trial 0 with value: 0.8027602070155262.
[I 2023-11-28 12:35:46,048] Trial 3 finished with value: 0.8016101207590569 and parameters: {'n_estimators': 188, 'min_samples_split': 10, 'min_samples_leaf': 7, 'max_features': 0.5

In [20]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

Best Score: 0.8079
Best params:  {'n_estimators': 85, 'min_samples_split': 2, 'min_samples_leaf': 5, 'max_features': 0.6193877243046408}


In [21]:
# model finalization
# 반드시 고정해둔 파라미터는 추가적으로 작성해줘야 한다.
print("Validation ACC")
best_params = study.best_params
best_params["max_depth"]=10
best_model = RandomForestClassifier(**best_params)
best_model.fit(X_train, y_train)
print("Validation Score : %.3f" % accuracy_score(y_val, best_model.predict(X_val)))

Validation ACC


RandomForestClassifier(max_depth=10, max_features=0.6193877243046408,
                       min_samples_leaf=5, n_estimators=85)

Validation Score : 0.803


## 7. 테스트 및 제출 파일 생성

In [22]:
group = test.PassengerId.apply(lambda x:x[:4])
temp = group.value_counts()
large_group_num = temp[temp >= 4].index
# isin이 0 혹은 1반환 
test['in_large_group'] = group.isin(large_group_num) * 1
# 의미있어보이지 않음..
test

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,in_large_group
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning,0
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers,0
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus,0
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter,0
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter,0
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron,0
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore,0
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale,0


In [23]:
# 결측치가 있는 column
# 결측치 제거 방법
# 1) cat_cols에 대해서는 mode를 계산해서 채워주고, num_cols에 대해서는 mean을 채워줌
test[cat_cols] = test[cat_cols].apply(lambda col: col.fillna(col.mode()[0]), axis=0) # mode()를 채울 땐 [0]을 추가해줘야 함.
test[num_cols.drop('Transported')] = test[num_cols.drop('Transported')].apply(lambda col: col.fillna(col.mean()), axis=0)
test.info()
# categorical feature encoding
test = pd.get_dummies(data=test, columns=['HomePlanet','Destination'])
test.drop(drop_cols, axis=1, inplace=True)
test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PassengerId     4277 non-null   object 
 1   HomePlanet      4277 non-null   object 
 2   CryoSleep       4277 non-null   bool   
 3   Cabin           4277 non-null   object 
 4   Destination     4277 non-null   object 
 5   Age             4277 non-null   float64
 6   VIP             4277 non-null   bool   
 7   RoomService     4277 non-null   float64
 8   FoodCourt       4277 non-null   float64
 9   ShoppingMall    4277 non-null   float64
 10  Spa             4277 non-null   float64
 11  VRDeck          4277 non-null   float64
 12  Name            4277 non-null   object 
 13  in_large_group  4277 non-null   int32  
dtypes: bool(2), float64(6), int32(1), object(5)
memory usage: 392.7+ KB


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,in_large_group,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,27.000000,False,0.0,0.0,0.0,0.0,0.0,0,True,False,False,False,False,True
1,False,19.000000,False,0.0,9.0,0.0,2823.0,0.0,0,True,False,False,False,False,True
2,True,31.000000,False,0.0,0.0,0.0,0.0,0.0,0,False,True,False,True,False,False
3,False,38.000000,False,0.0,6652.0,0.0,181.0,585.0,0,False,True,False,False,False,True
4,False,20.000000,False,10.0,0.0,635.0,0.0,0.0,0,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,True,34.000000,False,0.0,0.0,0.0,0.0,0.0,0,True,False,False,False,False,True
4273,False,42.000000,False,0.0,847.0,17.0,10.0,144.0,0,True,False,False,False,False,True
4274,True,28.658146,False,0.0,0.0,0.0,0.0,0.0,0,False,False,True,True,False,False
4275,False,28.658146,False,0.0,2680.0,0.0,0.0,523.0,0,False,True,False,False,False,True


In [24]:
X_test = test

preds = best_model.predict(X_test)
preds

array([ True, False,  True, ...,  True,  True, False])

In [25]:
submission['Transported'] = preds
submission.to_csv("submission/submission(23.11.28 rf).csv", index=False)